## spark sql

1-create df

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, max, sum, countDistinct

In [26]:
spark = SparkSession.builder.appName("lab3").getOrCreate()

df = spark.read.csv("ngram.csv", header=False, inferSchema=True, sep='\t')
df = df.toDF('bigram','year','count', 'pages','books')

df.printSchema()
df.show(10)

root
 |-- bigram: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- pages: integer (nullable = true)
 |-- books: integer (nullable = true)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
+--------+----+-----+-----+-----+
only showing top 10 rows



2-tmp table

In [27]:
df.createOrReplaceTempView("ngram")

3.1-return all bigrams where the Count is greater than five.

In [28]:
# sql
spark.sql("Select * from ngram where count > 5").show(10)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
+--------+----+-----+-----+-----+
only showing top 10 rows



In [29]:
# sparkSql
df.filter("count > 5").select("*").show(10)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
+--------+----+-----+-----+-----+
only showing top 10 rows



In [30]:
df.filter(df['count'] > 5).show(10)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
+--------+----+-----+-----+-----+
only showing top 10 rows



In [31]:
df.filter(col('count') > 5).show(10)

+--------+----+-----+-----+-----+
|  bigram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
+--------+----+-----+-----+-----+
only showing top 10 rows



3.2-return the total number of bigrams for each year.

In [32]:
# sql
spark.sql('select year, count(*) as total_bigram from ngram group by year').show(10)

+----+------------+
|year|total_bigram|
+----+------------+
|1829|           1|
|1990|           2|
|1903|           1|
|1884|           1|
|1888|           1|
|1924|           1|
|2003|           2|
|1823|           1|
|2007|           2|
|1869|           1|
+----+------------+
only showing top 10 rows



In [33]:
# sparkSql
df.groupBy('year').agg(count('*').alias('total_bigram')).show(10)

+----+------------+
|year|total_bigram|
+----+------------+
|1829|           1|
|1990|           2|
|1903|           1|
|1884|           1|
|1888|           1|
|1924|           1|
|2003|           2|
|1823|           1|
|2007|           2|
|1869|           1|
+----+------------+
only showing top 10 rows



3.3-return the bigrams with the highest Count for each year.

In [34]:
# sql
spark.sql('select year, bigram, MAX(count) as max_count from ngram group by year, bigram').show(10)

+----+--------+---------+
|year|  bigram|max_count|
+----+--------+---------+
|2002|    ! 09|        5|
|1932|    ! 09|        1|
|1880|    ! 09|        2|
|1935|    ! 09|        2|
|1999|    ! 09|       11|
|1829|    ! 09|        3|
|1999|! $17.95|       11|
|1887|    ! 09|        1|
|2007|    ! 09|        2|
|1899|    ! 09|        6|
+----+--------+---------+
only showing top 10 rows



In [35]:
# sparkSql
df.groupBy("year","bigram").max("count").show(10)

+----+--------+----------+
|year|  bigram|max(count)|
+----+--------+----------+
|2002|    ! 09|         5|
|1932|    ! 09|         1|
|1880|    ! 09|         2|
|1935|    ! 09|         2|
|1999|    ! 09|        11|
|1829|    ! 09|         3|
|1999|! $17.95|        11|
|1887|    ! 09|         1|
|2007|    ! 09|         2|
|1899|    ! 09|         6|
+----+--------+----------+
only showing top 10 rows



3.4-return all bigrams that appeared in 20 different years.

In [36]:
# sql
spark.sql('select bigram from ngram group by bigram having count(distinct year) = 20').show(10)

+--------+
|  bigram|
+--------+
|! $17.95|
+--------+



In [37]:
# sparkSql
df.groupby('bigram').agg(countDistinct('year').alias('distinct_year')).filter('distinct_year = 20').show(10)

+--------+-------------+
|  bigram|distinct_year|
+--------+-------------+
|! $17.95|           20|
+--------+-------------+



3.5-return all bigrams that contain the character '!' in the first part and the character '9' in the second part (the two parts are separated by a space).

In [46]:
# sql
spark.sql('select year, bigram from ngram where bigram like "%!% %9%"').show(10)

+----+--------+
|year|  bigram|
+----+--------+
|1985|! $17.95|
|1987|! $17.95|
|1990|! $17.95|
|1991|! $17.95|
|1992|! $17.95|
|1993|! $17.95|
|1995|! $17.95|
|1996|! $17.95|
|1997|! $17.95|
|1998|! $17.95|
+----+--------+
only showing top 10 rows



In [47]:
# sparkSql
df.filter('bigram like "%%! %9%"').select('year', 'bigram').show(10)

+----+--------+
|year|  bigram|
+----+--------+
|1985|! $17.95|
|1987|! $17.95|
|1990|! $17.95|
|1991|! $17.95|
|1992|! $17.95|
|1993|! $17.95|
|1995|! $17.95|
|1996|! $17.95|
|1997|! $17.95|
|1998|! $17.95|
+----+--------+
only showing top 10 rows



3.6-return the bigrams that appeared in all the years present in the dataset.

In [40]:
# sql
spark.sql('select bigram from ngram group by bigram having count(distinct year) = (select count(distinct year) from ngram)').show(10)

+------+
|bigram|
+------+
|  ! 09|
+------+



In [49]:
df.select(countDistinct('year')).collect()

[Row(count(DISTINCT year)=100)]

In [48]:
# sparkSql
df.groupby('bigram').agg(countDistinct('year').alias('distinct_year')).filter(col('distinct_year') == df.select(countDistinct('year')).collect()[0][0]).show(10)

+------+-------------+
|bigram|distinct_year|
+------+-------------+
|  ! 09|          100|
+------+-------------+



3.7-return the total number of pages and books in which each bigram appeared for each available year, sorted in alphabetical order.

In [42]:
# sql
spark.sql('select year, bigram, sum(pages) as total_pages, sum(books) as total_books from ngram group by year, bigram order by bigram').show(10)

+----+--------+-----------+-----------+
|year|  bigram|total_pages|total_books|
+----+--------+-----------+-----------+
|2002|! $17.95|          5|          5|
|1993|! $17.95|          2|          2|
|2006|! $17.95|          5|          5|
|1985|! $17.95|          1|          1|
|1996|! $17.95|          2|          2|
|1999|! $17.95|         10|         10|
|1995|! $17.95|          1|          1|
|2007|! $17.95|          2|          2|
|1991|! $17.95|          1|          1|
|2000|! $17.95|          9|          9|
+----+--------+-----------+-----------+
only showing top 10 rows



In [43]:
# sparkSql
df.groupby('year', 'bigram').agg(sum('pages').alias('total_pages'), sum('books').alias('total_books')).orderBy('bigram').show(10)

+----+--------+-----------+-----------+
|year|  bigram|total_pages|total_books|
+----+--------+-----------+-----------+
|2002|! $17.95|          5|          5|
|1993|! $17.95|          2|          2|
|2006|! $17.95|          5|          5|
|1985|! $17.95|          1|          1|
|1996|! $17.95|          2|          2|
|1999|! $17.95|         10|         10|
|1995|! $17.95|          1|          1|
|2007|! $17.95|          2|          2|
|1991|! $17.95|          1|          1|
|2000|! $17.95|          9|          9|
+----+--------+-----------+-----------+
only showing top 10 rows



3.8-return the total number of distinct bigrams for each year, sorted in descending order of the year.

In [44]:
# sql
spark.sql('select year, count(distinct bigram) as distinct_bigram from ngram group by year order by year desc').show(10)

+----+---------------+
|year|distinct_bigram|
+----+---------------+
|2008|              2|
|2007|              2|
|2006|              2|
|2005|              2|
|2004|              2|
|2003|              2|
|2002|              2|
|2001|              2|
|2000|              2|
|1999|              2|
+----+---------------+
only showing top 10 rows



In [45]:
df.groupBy('year').agg(countDistinct('bigram').alias('distinct_bigram')).orderBy('year', ascending=False).show(10)

+----+---------------+
|year|distinct_bigram|
+----+---------------+
|2008|              2|
|2007|              2|
|2006|              2|
|2005|              2|
|2004|              2|
|2003|              2|
|2002|              2|
|2001|              2|
|2000|              2|
|1999|              2|
+----+---------------+
only showing top 10 rows

